# Indexes

Table indexes are data structures that allow fast lookups by an indexed attribute or combination of attributes.

In DataJoint, indexes are created by one of the three mechanisms:

1. Primary key 
2. Foreign key 
3. Explicitly defined indexes

The first two mechanisms are obligatory. Every table has a primary key, which serves as an unique index. Therefore, restrictions by a primary key are very fast. Foreign keys create additional indexes unless a suitable index already exists.

Let's test this principle. Let's create a table with a 10,000 entries and compare lookup times:

In [1]:
import datajoint as dj
schema = dj.schema('test_indexes')
schema.drop() # drop previous schema definition (if any) and create anew
schema = dj.schema('test_indexes')

Connecting dimitri@db.ust-db.link:3306
Proceed to delete entire schema `test_indexes`? [yes, No]: yes


Let's say a mouse in the lab has a lab-specific ID but it also has a separate id issued by the animal facility.

In [2]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    """

In [3]:
import random
def populate_mice(n=200_000):
    """insert a bunch of mice"""
    Mouse.insert(
        ((random.randint(1,1000_000_000), random.randint(1,1000_000_000)) 
         for i in range(n)), skip_duplicates=True)

In [4]:
populate_mice()

In [5]:
Mouse()

mouse_id lab-specific ID,tag_id animal facility ID
4011,342875954
9419,879720862
12546,334177269
17801,25418429
19413,312316071
22506,754941865
28086,957278686
31092,560024909
38208,85479847
40689,186813683


In [7]:
%%timeit -n6 -r3

# efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

38.1 ms ± 1.44 ms per loop (mean ± std. dev. of 3 runs, 6 loops each)


In [9]:
%%timeit -n3 -r3

# inefficient! Requires a full table scan
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

98.9 ms ± 5.02 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


The indexed searches are much faster!

To make searches faster on fields other than the primary key or a foreign key, you can add a secondary index explicitly. 

Regular indexes are declared as `index(attr1, ..., attrN)` on a separate line anywhere in the table declration (below the primary key divide). 

Indexes can be declared with unique constraint as  `unique index (attr1, ..., attrN)`.

Let's redeclare the table with a unique index on `tag_id`.

In [10]:
Mouse.drop()

`test_indexes`.`mouse` (199980 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [11]:
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id : int  # lab-specific ID
    ---
    tag_id : int  # animal facility ID
    unique index (tag_id)
    """

In [12]:
populate_mice()

In [13]:
Mouse()

mouse_id lab-specific ID,tag_id animal facility ID
445133168,3412
407589124,4308
872887958,4663
971995456,5643
285299809,7603
217671106,10593
621030453,11647
447254071,13795
592162441,20127
15064041,20699


Now both types of searches are equally efficient!

In [15]:
%%timeit -n3 -r3

#efficient! Uses the primary key
(Mouse() & {'mouse_id': random.randint(0, 999_999)}).fetch()

43 ms ± 6.77 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [16]:
%%timeit -n6 -r6

#efficient! Uses the seconary index on tag_id
(Mouse() & {'tag_id': random.randint(0, 999_999)}).fetch()

40.7 ms ± 8.19 ms per loop (mean ± std. dev. of 6 runs, 6 loops each)


Let's now imagine that rats in the `Rat` table are identified by the combination of lab the `lab_name` and `rat_id` in each lab:

In [17]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date
    """

In [18]:
def populate_rats():
    lab_names = ("Cajal", "Kandel", "Moser", "Wiesel")
    for date_of_birth in (None, "2019-10-01", "2019-10-02", "2019-10-03", "2019-10-04"):
        Rat.insert((
            (random.choice(lab_names), random.randint(1, 1_000_000), date_of_birth) 
                   for i in range(100_000)), skip_duplicates=True)

In [19]:
populate_rats()

In [20]:
Rat()

lab_name,rat_id lab-specific ID,date_of_birth
Cajal,4,None
Cajal,43,2019-10-03
Cajal,57,2019-10-01
Cajal,60,2019-10-03
Cajal,64,2019-10-04
Cajal,70,2019-10-02
Cajal,71,None
Cajal,76,2019-10-02
Cajal,88,2019-10-01
Cajal,100,None


Note that dispite the fact that `rat_id` is in the index, search by `rat_id` alone are not helped by the index because it is not first in the index. This is similar to search for a word in a dictionary that orders words alphabetically. Searching by the first letters of a word is easy but searching by the last few letters of a word requires scanning the whole dictionary.

In this table, the primary key is a unique index on the combination `(lab_id, rat_id)`. Therefore searches on these attributes or on `lab_id` alone are fast. But this index cannot help searches on `rat_id` alone:

In [21]:
%%timeit -n2 -r10

# inefficient!  Requires full table scan.
(Rat() & {'rat_id': 300}).fetch()

191 ms ± 43.8 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [22]:
%%timeit -n2 -r10

# efficient! Uses the primary key
(Rat() & {'rat_id': 300, 'lab_name': 'Cajal'}).fetch()

43.7 ms ± 20.8 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [ ]:
%%timeit -n2 -r10

# inefficient! Requires a full table scan
len(Rat & {'rat_id': 500})

Pattern searches in strings can benefit from an index when the starting characters are specified.

In [28]:
%%timeit -n2 -r2

# efficient! Uses the primary key
(Rat & 'lab_name LIKE "Caj%"').fetch()

1.73 s ± 69.9 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [29]:
%%timeit -n2 -r2

# inefficient! requires a full table scan
(Rat & 'lab_name LIKE "%jal"').fetch()

1.83 s ± 37.9 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Similarly, searching by the date requires an inefficient full-table scan:

In [32]:
%%timeit -n2 -r2

len(Rat & 'date_of_birth > "2019-10-02"')

562 ms ± 106 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


To speed up searches by the `rat_id` and `date_of_birth`, we can explicit indexes to `Rat`:

In [33]:
Rat.drop()

`test_indexes`.`rat` (470183 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [34]:
@schema
class Rat(dj.Manual):
    definition = """
    lab_name : char(16) 
    rat_id : int unsigned # lab-specific ID
    ---
    date_of_birth = null : date

    index(rat_id)
    index(date_of_birth)
    """

In [35]:
populate_rats()

In [36]:
%%timeit -n2 -r10

# efficient!  uses index on rat_id
(Rat() & {'rat_id': 300}).fetch()

37.1 ms ± 471 µs per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [37]:
%%timeit -n2 -r2

# efficient! uses index on date_of_birth
len(Rat & 'date_of_birth > "2019-10-02"')

309 ms ± 9.72 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


#### Quiz: How many indexes does the table `Rat` have?

In [ ]:
Rat.describe();

#### Answer


Three: primary key, rat_id, date_of_birth

# Indexes in SQL

In [ ]:
import json
import pymysql 

pymysql.install_as_MySQLdb()

with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

In [ ]:
%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

In [ ]:
%%sql

create database dimitri_indexes

In [ ]:
%%sql

SHOW TABLES in dimitri_indexes;

In [ ]:
%%sql

CREATE TABLE dimitri_indexes.mouse(
mouse_id  int  NOT NULL,
tag_id int NOT NULL,
primary key(mouse_id)
)

In [ ]:
%%sql

drop table dimitri_indexes.mouse

In [ ]:
%%sql

CREATE TABLE dimitri_indexes.mouse(
mouse_id  int  NOT NULL,
tag_id int NOT NULL,
primary key(mouse_id),
index (tag_id)
)

In [ ]:
%%sql

CREATE UNIQUE INDEX mouse_idx ON dimitri_indexes.mouse (tag_id)

In [ ]:
%%sql

DROP KEY mouse_idx